In [1]:
import pandas as pd
import numpy as np
import math
import copy as cp
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz

train_df = pd.read_csv("./data/train_final.csv")
m_train = len(train_df)
test_df = pd.read_csv("./data/test_final.csv")
m_test = len(test_df)
features = list(train_df.columns[:14])
print("* features:", features, sep="\n")

* features:
['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'capital.gain', 'capital.loss', 'hours.per.week', 'native.country']


# preprocess the data
- workclass, occupation, native.country has anomaly data '?'

In [2]:


categorical_attrib_values = { "workclass":['Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov', 'State-gov', 'Without-pay', 'Never-worked'],\
                 "education":['Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', '9th', '7th-8th', '12th', 'Masters', '1st-4th', '10th', 'Doctorate', '5th-6th', 'Preschool'],\
                 "marital.status":['Married-civ-spouse', 'Divorced', 'Never-married', 'Separated', 'Widowed', 'Married-spouse-absent', 'Married-AF-spouse'],\
                 "occupation":['Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Armed-Forces'],\
                 "relationship":['Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative', 'Unmarried'],\
                 "race":['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black'],\
                 "sex":['Female', 'Male'],\
                 "native.country": ['United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada', 'Germany', 'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece', 'South', 'China', 'Cuba', 'Iran', 'Honduras', 'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam', 'Mexico', 'Portugal', 'Ireland', 'France', 'Dominican-Republic', 'Laos', 'Ecuador', 'Taiwan', 'Haiti', 'Columbia', 'Hungary', 'Guatemala', 'Nicaragua', 'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago', 'Peru', 'Hong', 'Holand-Netherlands']
                }
def convert_numerical_to_binary(df):
    for col_name in df.columns:
        if col_name not in categorical_attrib_values:
            df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
            threashold = df[col_name].median()
            print("preprocessing numerical attribute:", col_name, "threashold:", threashold)
            df[col_name] = df[col_name].apply(lambda x: 0 if int(x) <= threashold else 1)

######## convert the numerical to binary
train_df_processed = train_df
test_df_processed = test_df
convert_numerical_to_binary(train_df_processed)
convert_numerical_to_binary(test_df_processed)

preprocessing numerical attribute: age threashold: 37.0
preprocessing numerical attribute: fnlwgt threashold: 177299.5
preprocessing numerical attribute: education.num threashold: 10.0
preprocessing numerical attribute: capital.gain threashold: 0.0
preprocessing numerical attribute: capital.loss threashold: 0.0
preprocessing numerical attribute: hours.per.week threashold: 40.0
preprocessing numerical attribute: income>50K threashold: 0.0
preprocessing numerical attribute: ID threashold: 11921.5
preprocessing numerical attribute: age threashold: 37.0
preprocessing numerical attribute: fnlwgt threashold: 179266.5
preprocessing numerical attribute: education.num threashold: 10.0
preprocessing numerical attribute: capital.gain threashold: 0.0
preprocessing numerical attribute: capital.loss threashold: 0.0
preprocessing numerical attribute: hours.per.week threashold: 40.0


# fit a tree

In [3]:
y = train_df["income>50K"]
X = pd.get_dummies(train_df_processed[features],drop_first=True)
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)
dt.fit(X, y)

DecisionTreeClassifier(min_samples_split=20, random_state=99)

In [4]:
incorrect = np.count_nonzero(dt.predict(X)-y)
print("train_accuracy", 1 - incorrect/X.shape[0])

train_accuracy 0.87976


In [9]:
X_test = pd.get_dummies(test_df_processed[features],drop_first=True)